In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotnine as pn
import seaborn as sns 
import torch

import utils
from sklearn.metrics.pairwise import cosine_similarity, paired_distances
from scipy.spatial import distance
from alive_progress import alive_bar


In [ ]:
import importlib
importlib.reload(utils)


In [ ]:
# subj = utils.ExpInfo.getSubjIDs()[1]
subj = 'K-Reg-H-7'
task = utils.ExpInfo.taskName[1]


In [ ]:
# xhy = utils.LoadData.xhy(subj, task)
# xhy_disp = utils.LoadData.xhy_disp(subj, task)
d, d_disp = utils.LoadData.mouseMovement_array(subj, task, velocity=True)
df_beh = utils.LoadData.behaviorData(subj, task) 

In [6]:
iTrial = 3
wSize = 100

dist_measure = 'euclidean'
# target = df_beh.loc[iTrial, 'target dot']
# choice = df_beh.loc[iTrial, 'selected dot']




In [11]:
dists = []
dist_coss = []
with alive_bar(len(d), force_tty=True) as bar:
    for iTrial in range(len(d)):

        # ---------------------------------------------------------------------------- #
        #                                compute forward                               #
        # ---------------------------------------------------------------------------- #
        x = d[iTrial]
        x = utils.DataProcessing.rollingWindow(x, wSize, 1)
        h, y = utils.test.quick_forward(subj, x)

        h_disp = []
        for i in range(3):
            x_disp = d_disp[iTrial][:, i*2:i*2+2]
            x_disp = utils.DataProcessing.rollingWindow(x_disp, wSize, 1)
            h_disp_, y_disp = utils.test.quick_forward(subj, x_disp)
            h_disp.append(h_disp_)


        # ---------------------------------------------------------------------------- #
        #                                   distance                                   #
        # ---------------------------------------------------------------------------- #
        dist_cos = []
        dist = []
        for iDot in range(3):
            h1 = h
            h2 = h_disp[iDot]
            dist_cos_ = cosine_similarity(h1, h2).diagonal()
            dist_ = paired_distances(h1, h2, metric=dist_measure)
            dist_cos.append(dist_cos_)
            dist.append(dist_)
            
        dist = np.vstack(dist).T
        dist_cos = np.vstack(dist_cos).T
        
        dists.append(dist)
        dist_coss.append(dist_cos)
        bar()


|▋⚠︎                                      | (!) 1/60 [2%] in 42.7s (0.02/s)                                              


In [12]:
df_beh_ = df_beh.copy()
df_beh_['dist'] = dist


ValueError: Length of values (200) does not match length of index (60)

# ROC

In [ ]:
# from sklearn import metrics 

# y_true = np.zeros(dist.shape)
# y_true[:, 1] = 1
# fpr, tpr, thresholds = metrics.roc_curve(y_true.flatten(), 1-dist.flatten())
# roc_auc = metrics.auc(fpr, tpr)
# metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc).plot()

# y_true = np.zeros(dist.shape)
# target = df_beh['target dot'][iTrial]-1
# y_true[:, target] = 1
# # fpr, tpr, thresholds = metrics.roc_curve(y_true.flatten(), 1-dist.flatten())
# fpr, tpr, thresholds = metrics.roc_curve(y_true.flatten(), 1-dist.flatten())
# roc_auc.append(metrics.auc(fpr, tpr))

# xy space

In [ ]:
# d1 = d[0][iTrial]
# dist = []
# for i in range(3):
#     d2 = d[1][iTrial][:, i*2:i*2+2]
#     dist_ = cosine_similarity(d1, d2).diagonal()
#     # dist_ = paired_distances(d1, d2, metric=dist_measure)
#     dist.append(dist_)
# dist = np.vstack(dist).T

# fig, ax = plt.subplots(1, 1, figsize=(10, 5))
# ax.plot(dist)
# ax.legend(['dot 1', 'dot 2', 'dot 3'])

# fig, ax = plt.subplots(1, 1, figsize=(10, 5))
# ax.plot(dist.cumsum(axis=0))
# ax.legend(['dot 1', 'dot 2', 'dot 3'])
